# Example: Turbofan Engine Degradation Simulation Data Set

This uses the dataset explored in Dayne Batten's implementation of WTTE-RNN in Keras.

Reference:
```
A. Saxena and K. Goebel (2008). 
"Turbofan Engine Degradation Simulation Data Set", 
https://ti.arc.nasa.gov/c/13/, NASA Ames, Moffett Field, CA.
```

In [ ]:
from wtte.network import WtteRnnNetwork
from wtte.loss import loss_continuous_weibull_loglik, loss_discrete_weibull_loglik
from wtte.datasets import TurbofanDegradationDataset
from wtte.train import train
from wtte.predict import predict
from wtte.visualize import plot_weibull_predictions, plot_predictions_over_time

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

In [ ]:
# Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.deterministic = True  # Fix for CUDA unspecified launch failure

## Train model

In [ ]:
dataset_train = TurbofanDegradationDataset(directory='D:/Users/Aaron/Documents/AI and ML Projects/Data/CMAPSS',
                                           train=True, min_seq_len=20, max_seq_len=100,
                                           unit_ids=[1,2,3,4])

logging.info(len(dataset_train))

In [ ]:
dataset_test = TurbofanDegradationDataset(directory='D:/Users/Aaron/Documents/AI and ML Projects/Data/CMAPSS',
                                           train=False, min_seq_len=100, max_seq_len=100,
                                           unit_ids=[1])
dataset_test.standardize(dataset_train)

logging.info(len(dataset_test))

In [ ]:
dl_train = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=5, pin_memory=True,
                      collate_fn=dataset_train.collate_fn)

In [ ]:
dl_test = DataLoader(dataset_test, batch_size=128, shuffle=False, num_workers=5, pin_memory=True,
                      collate_fn=dataset_train.collate_fn)

In [ ]:
model = WtteRnnNetwork(input_size=len(dataset_train.features), 
                       rnn_layer=nn.LSTM, 
                       rnn_layer_options={'hidden_size': 25, 'num_layers': 2, 'dropout': 0.25}) \
                           .to(device)

In [ ]:
train_model = True
model_filepath = 'D:/Users/Aaron/Documents/AI and ML Projects/My Projects/Predictive Maintenance/wtte_rnn_2.pt'

if train_model:
    train(model, dl_train, test_dataloader=dl_test, n_epochs=200, lr=0.01, clip_grad=1.0, 
          loss_type='discrete', device=device, n_epochs_pretrain=10)
    # Save trained model
    torch.save(model.state_dict(), model_filepath)
else:
    # Load trained model
    model.load_state_dict(torch.load(model_filepath))

## Predict model

In [ ]:
# Just one sequence
test_seq_indices = [0]
dl_test_2 = DataLoader(Subset(dataset_test, test_seq_indices), 
                       batch_size=min(len(test_seq_indices), 1024), 
                       shuffle=False, pin_memory=True,
                       collate_fn=dataset_train.collate_fn)

test_output = predict(model, dl_test_2, device=device, to_dataframe=True)

In [ ]:
plt.rcParams["figure.figsize"] = (8,6)
plot_predictions_over_time(test_output)

In [ ]:
plt.rcParams["figure.figsize"] = (8,15)
plot_weibull_predictions(test_output, sample_frac=0.10)